In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math

%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
def exponential(nsamples,rate):
    x = np.zeros(nsamples)
    u = np.random.samples(nsamples)
    for i in range(nsamples):
        x[i] = - math.log(1.0-u[i])/rate
    return x

In [ ]:
# defined by a non-homogenous Poisson process
def nextArrival(t,rate,ratemax):
    while(1):
        Z = exponential(1,ratemax)[0]
        t = t + Z
        U = np.random.sample(1)
        if U < rate(t) / ratemax:
            return t

In [ ]:
price = 10 * np.random.samples(1)[0] + 1

def cost(y): 
    return y * price

In [1]:
def inventory(G,T,S,s,c,x0,r,L,h,rate,ratemax):
    
    C = 0 # total amount of ordering costs
    R = 0 # total amount of revenue earned
    H = 0 # total amount of inventory holding costs
    x = x0 # amount of inventory
    y = 0 # amount on order
    t0 = nextArrival(rate,ratemax,T) # arrival time of next customer
    t1 = 1.0e+30 # time of order delivery
    t = 0.0
    
    while(1):
        
        # customer arrives before delivery
        if t0 < t1 and t < T:
            H = H + (t0 - t) * x * h
            t = t0
            D = G() 
            w = min(D,x) # amount of the order that can be filled
            R = R + w * r
            x = x - w
            if x < s and y == 0:
                y = S - x
                t1 = t + L
            U = np.random.sample(1)[0]
            t0 = t - (1/rate) * math.log(U)

        elif t0 >= t1 and t < T:
            H = H + (t1 - t) * x * h
            t = t1
            C = C + c(y)
            x = x + y
            y = 0
            t1 = 1.0e+30
            
        else:
            profit = (R - C - H)/T # profit per unit time
            return profit

Hello


In [ ]:
inventory(G,T,S,s,exponential,x0,r,L,rate,ratemax)